In [1]:
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
import random
import sys 
from selenium_stealth import stealth
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')

import tkinter as tk
def on_submit():
    global user_id, user_pw, customer_ids
    user_id = entry_user.get().strip()
    user_pw = entry_pw.get().strip()
    raw = entry_cust.get().rstrip(',')
    customer_ids = [cid.strip() for cid in raw.split(",") if cid.strip()]
    root.destroy()

def on_close():
    """창 닫기(X) 클릭 시 프로그램 완전 종료"""
    root.destroy()
    sys.exit()

# ─── TKinter 입력 창 ─────────────────────────────────────────────────────────
root = tk.Tk()
root.title("네이버 로그인 및 Customer ID 입력")
root.geometry("350x240")
root.resizable(False, False)

# WM_DELETE_WINDOW 프로토콜에 on_close 연결
root.protocol("WM_DELETE_WINDOW", on_close)

tk.Label(root, text="네이버 아이디:", anchor="w").pack(fill="x", padx=10, pady=(10,0))
entry_user = tk.Entry(root)
entry_user.pack(fill="x", padx=10)

tk.Label(root, text="네이버 비밀번호:", anchor="w").pack(fill="x", padx=10, pady=(10,0))
entry_pw = tk.Entry(root, show="*")
entry_pw.pack(fill="x", padx=10)

tk.Label(root, text="Customer IDs (ID 입력 후 ENTER):", anchor="w").pack(fill="x", padx=10, pady=(10,0))
entry_cust = tk.Entry(root)
entry_cust.pack(fill="x", padx=10)

def on_cust_enter(event):
    if entry_cust.get().strip() and not entry_cust.get().endswith(','):
        entry_cust.insert(tk.END, ',')
    return 'break'

entry_cust.bind('<Return>', on_cust_enter)

btn = tk.Button(root, text="OK", command=on_submit)
btn.pack(pady=15)

root.mainloop()
# ─────────────────────────────────────────────────────────────────────────────

# ChromeOptions 설정
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지

def human_like_scroll(driver,
                      down_min_step=100, down_max_step=200,
                      up_min_step=150, up_max_step=500,
                      pause_min=0.1, pause_max=0.3,
                      bottom_pause_min=1.0, bottom_pause_max=2.0):
    total_height = driver.execute_script("return document.body.scrollHeight;")
    current_y = 0

    # (2) ↓ 내려갈 때: 작은 랜덤 스텝
    while current_y < total_height:
        step = random.randint(down_min_step, down_max_step)
        driver.execute_script("window.scrollBy(0, arguments[0]);", step)
        current_y = min(current_y + step, total_height)
        time.sleep(random.uniform(pause_min, pause_max))

    # (3) 바닥에서 잠시 휴식
    time.sleep(random.uniform(bottom_pause_min, bottom_pause_max))

    # (4) ↑ 올라올 때: 큰 랜덤 스텝
    while current_y > 0:
        step = random.randint(up_min_step, up_max_step)
        driver.execute_script("window.scrollBy(0, -arguments[0]);", step)
        current_y = max(current_y - step, 0)
        time.sleep(random.uniform(pause_min, pause_max))

def human_like_move(driver, element):
    actions = ActionChains(driver)
    # 1) 요소 중앙으로 이동
    actions.move_to_element(element)
    # 2) 살짝 흔들며 자연스럽게
    offset_x = random.randint(-5, 5)
    offset_y = random.randint(-5, 5)
    actions.move_by_offset(offset_x, offset_y)
    # 3) 잠깐 머뭄
    actions.pause(random.uniform(0.2, 0.5))
    actions.perform()

def random_delay(min_s, max_s):
    time.sleep(random.uniform(min_s, max_s))

driver = webdriver.Chrome(options = options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

stealth(driver,
        languages=["ko-KR", "ko"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
)
url = "http://www.naver.com"
driver.get(url)
time.sleep(2)

# user_id = 'rooda7'
# user_pw = 'dlstk1234!@'

human_like_scroll(driver)
random_delay(0.5, 1.0)
elem = driver.find_element(by=By.CLASS_NAME, value="MyView-module__link_login___HpHMW")
human_like_move(driver, elem)
elem.click()
random_delay(0.5, 1.2)

human_like_scroll(driver)
log_ID = driver.find_element(by=By.ID, value="id")
log_ID.click()
human_like_move(driver, log_ID)
random_delay(0.1, 0.3)
pyperclip.copy(user_id)
log_ID.send_keys(Keys.CONTROL, 'v')

human_like_scroll(driver)
time.sleep(1)
log_PID = driver.find_element(by=By.ID, value="pw")
human_like_move(driver, log_PID)
log_PID.click()
random_delay(0.1, 0.3)
pyperclip.copy(user_pw)
log_PID.send_keys(Keys.CONTROL, 'v')
time.sleep(2)

human_like_move(driver, log_PID)
random_delay(0.1, 0.3)
log_ENT = driver.find_element(by=By.ID, value="log.login")
human_like_move(driver, log_ENT)
random_delay(0.1, 0.3)
log_ENT.click()
time.sleep(2)

human_like_scroll(driver)
time.sleep(1)

wait = WebDriverWait(driver, 10)

dont_save_button = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="new.dontsave"]')
))
dont_save_button.click()

time.sleep(2)
human_like_scroll(driver)
random_delay(0.5, 1.0)
search_param = driver.find_element(by=By.ID,value = 'query')
human_like_move(driver, search_param)
random_delay(0.1, 0.3)
serch_input = '네이버검색광고'
pyperclip.copy(serch_input)
search_param.send_keys(Keys.CONTROL,'v')
search_click = driver.find_element(by=By.ID,value = 'search-btn')
search_click.click()
time.sleep(2)

검색광고 링크클릭

In [2]:
link_click = driver.find_element(
    By.CSS_SELECTOR,
    "span.sds-comps-text.sds-comps-text-type-body2.sds-comps-text-weight-sm"
)
human_like_move(driver, link_click)
random_delay(0.5, 1.0)
link_click.click()

In [3]:
time.sleep(2)  
original = driver.current_window_handle
wait.until(lambda d: len(d.window_handles) > 1)
new_window = [h for h in driver.window_handles if h != original][0]
driver.switch_to.window(new_window)
print("▶ Switched to new window:", new_window)

▶ Switched to new window: AEB7AF3FEFB484BF38D4C382491FAAC0


In [4]:
human_like_scroll(driver)
random_delay(0.5, 1.0)
click2 = driver.find_element(by=By.XPATH,value = '/html/body/marvel-root/membership-wrap/div/div/membership-select-account/div/div/div/div[3]/div/div[1]/a')
human_like_move(driver, click2)
random_delay(0.1, 0.3)
click2.click()
time.sleep(2)

human_like_scroll(driver)
random_delay(0.5, 1.0) 

click3 = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="container"]/my-screen/div/div[1]/div/my-screen-board/div/div[1]/ul/li[1]/a/span')
))
human_like_move(driver, click3)
random_delay(0.1, 0.3)
click3.click()
time.sleep(2)

In [5]:
original = driver.current_window_handle
wait.until(lambda d: len(d.window_handles) > 1)
new_window = [h for h in driver.window_handles if h != original][0]
driver.switch_to.window(new_window)
time.sleep(3)

original = driver.current_window_handle

human_like_scroll(driver)
random_delay(0.5, 1.0) 

# 모든 창 핸들을 가져오기
all_windows = driver.window_handles

# 원래 창을 제외한 나머지 창들 (뒤에 연 창들)
other_windows = [w for w in all_windows if w != original]

# 다른 창들 닫기
for w in other_windows:
    driver.switch_to.window(w)
    driver.close()

# 원래 창으로 다시 전환
driver.switch_to.window(original)
random_delay(1.0, 2.0)

human_like_scroll(driver)
random_delay(1.0, 2.0)

link_click = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, 'span.sds-comps-text.sds-comps-text-type-body2.sds-comps-text-weight-sm')
))
human_like_move(driver, link_click)
random_delay(0.5, 1.0)
link_click.click()

human_like_scroll(driver)
random_delay(0.3, 1.8) 
original = driver.current_window_handle
wait.until(lambda d: len(d.window_handles) > 1)
new_window = [h for h in driver.window_handles if h != original][0]
driver.switch_to.window(new_window)
print("▶ Switched to new window:", new_window)
random_delay(2.0, 3.0) 

click2 = driver.find_element(by=By.XPATH,value = '//*[@id="container"]/my-screen/div/div[1]/div/my-screen-board/div/div[1]/ul/li[1]/a/span')
human_like_move(driver, click2)
random_delay(0.1, 0.3)
click2.click()
random_delay(2.0, 3.0) 

▶ Switched to new window: 5D40BD518C075FB5F44B69A9F0C32250


In [6]:
wait = WebDriverWait(driver, 15)
random_delay(1.0, 2.0) 
original = driver.current_window_handle
wait.until(lambda d: len(d.window_handles) > 1)
new_window = [h for h in driver.window_handles if h != original][0]
driver.switch_to.window(new_window)
print("▶ Switched to new window:", new_window)
random_delay(1.0, 2.0) 

new_window = [h for h in driver.window_handles if h != original][1]
driver.switch_to.window(new_window)
print("▶ Switched to new window:", new_window)

▶ Switched to new window: 700DCC7CA8EB2C846747E168FB866C91
▶ Switched to new window: 357CC85890ADB6D25A7015A7B80EE47C


In [7]:
report_bt = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[2]/div/div/div[1]/ul/li[3]/div/a/span')
))
human_like_move(driver, report_bt)
random_delay(0.5, 1.0)
report_bt.click()

human_like_scroll(driver)
time.sleep(2)

wait.until(EC.visibility_of_element_located(
    (By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[2]/div/div/div[1]/ul/li[3]/div/div')
))
report_btt = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[2]/div/div/div[1]/ul/li[3]/div/div/div[1]/a/button')
))
human_like_move(driver, report_btt)
random_delay(0.1, 0.3)
report_btt.click()
random_delay(1.0, 2.0)

In [8]:
# def get_current_customer_display(driver):
#     """
#     상단에 현재 선택된 Customer ID 읽어오기
#     """
#     try:
#         CUSTOMER_ID_DISPLAY_XPATH = '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[1]/div[2]/div[1]/button/div/span/span[3]/em'
#         elem = driver.find_element(By.XPATH, CUSTOMER_ID_DISPLAY_XPATH)
#         return elem.text.strip()
#     except NoSuchElementException:
#         return None

# #  보고서 3개 다운로드
# def download_all_reports(driver, wait):
#     for row_index in [1, 2, 3]:
#         try:
#             print(f"[INFO] 보고서 클릭 → row {row_index}")
#             xpath = f'//*[@id="root"]/div[2]/div[2]/div/div/div[4]/div/div[3]/table/tbody/tr[{row_index}]/td[2]/a'
#             report_click = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
#             human_like_move(driver, report_click)
#             random_delay(1.0, 2.0)
#             report_click.click()
#             random_delay(1.0, 2.0)

#             human_like_scroll(driver)
#             random_delay(1.0, 2.0)
            
#             download_bt_xpath = '//*[@id="root"]/div[2]/div[2]/div/div/div[1]/div[2]/button'
#             download_bt = wait.until(EC.element_to_be_clickable((By.XPATH, download_bt_xpath)))
#             human_like_move(driver, download_bt)
#             random_delay(1.0, 2.0)
#             download_bt.click()
#             random_delay(1.0, 2.0)

#             driver.back()
#             random_delay(1.5, 2.0)

#         except TimeoutException:
#             print(f"[WARN] row {row_index}에 보고서 없음 → 다음 row로 이동")
#         except Exception as e:
#             print(f"[ERROR] row {row_index} 처리 중 오류: {e}")


# def change_customer_id(driver, wait, customer_id):
#     """
#     Customer ID 전환 시도 → 상단 표시된 ID가 바뀌었는지 비교 → 실패시 입력창 clear
#     """
#     previous_id = get_current_customer_display(driver)
#     print(f"[INFO] 전환 전 표시 ID: {previous_id}")

#     human_like_scroll(driver)
#     random_delay(0.5, 1.0)
#     re_cusid_input = wait.until(EC.element_to_be_clickable((
#         By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[1]/div[2]/div[1]/button/div')))
#     human_like_move(driver, re_cusid_input)
#     random_delay(0.1, 0.3)
#     re_cusid_input.click()
#     random_delay(0.1, 0.3)
#     cus_input = driver.find_element(By.CLASS_NAME, 'form-control')
#     human_like_move(driver, cus_input)
#     random_delay(0.5, 1.8)
#     cus_input.click()
#     random_delay(0.5, 1.8)
#     cus_input.send_keys(Keys.TAB)
#     random_delay(0.5, 1.8)

#     human_like_scroll(driver)
#     random_delay(0.5, 1.8)
    
#     customer_input = driver.switch_to.active_element
#     customer_input.clear()
#     random_delay(0.5, 1.8)

    
#     pyperclip.copy(customer_id)
#     customer_input.send_keys(Keys.CONTROL, 'v')
#     human_like_move(driver, customer_input)
#     random_delay(0.5, 1.8)
#     customer_input.send_keys(Keys.ENTER)
#     random_delay(0.5, 1.8)

#     new_id = get_current_customer_display(driver)
#     print(f"[INFO] 전환 후 표시 ID: {new_id}")
#     time.sleep(3)

#     if new_id and (customer_id in new_id) and (new_id != previous_id):
#         print(f"[SUCCESS] CustomerID {customer_id} 전환 성공!")
#         return True
#     else:
#         print(f"[FAIL] CustomerID {customer_id} 전환 실패")

#         try:
#             print("[INFO] 잘못된 입력값 clear 시도")
#             human_like_scroll(driver)
#             random_delay(0.5, 1.0)
#             re_cusid_input = wait.until(EC.element_to_be_clickable((
#                 By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[1]/div[2]/div[1]/button/div')))
#             human_like_move(driver, re_cusid_input)
#             random_delay(0.1, 0.3)
#             re_cusid_input.click()
#             random_delay(0.5, 1.8)

#             human_like_scroll(driver)
#             random_delay(0.5, 1.8)
            
#             cus_input = driver.find_element(By.CLASS_NAME, 'form-control')
#             cus_input.click()
#             random_delay(0.5, 1.8)
#             cus_input.send_keys(Keys.TAB)
#             random_delay(0.5, 1.8)

            
#             customer_input = driver.switch_to.active_element
#             customer_input.clear()
#             random_delay(1.3, 2.0)
#             print("[INFO] 입력창 clear 완료")
#         except Exception as e:
#             print(f"[WARN] 입력창 clear 실패: {e}")

#         return False

# #  메인 실행
# wait = WebDriverWait(driver, 15)

# # 전환할 Customer ID 리스트
# # customer_ids = ['2387950', '2910307', '825373']


# for cid in customer_ids:
#     print(f"\n=== [INFO] 전환 시도 → CustomerID: {cid} ===")
#     success = change_customer_id(driver, wait, cid)

#     if success:
#         download_all_reports(driver, wait)
#     else:
#         print(f"[SKIP] {cid} 전환 실패 → 다운로드 건너뜀")

In [9]:
def get_current_customer_display(driver):
    """
    상단에 현재 선택된 Customer ID 읽어오기
    """
    try:
        CUSTOMER_ID_DISPLAY_XPATH = '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[1]/div[2]/div[1]/button/div/span/span[3]/em'
        elem = driver.find_element(By.XPATH, CUSTOMER_ID_DISPLAY_XPATH)
        return elem.text.strip()
    except NoSuchElementException:
        return None

#  보고서 3개 다운로드
def download_all_reports(driver, wait):
    """
    보고서 3개 다운로드.
    1번 보고서가 없으면 False 반환해서 다음 고객으로 건너뜀.
    """
    for row_index in [1, 2, 3]:
        try:
            print(f"[INFO] 보고서 클릭 → row {row_index}")
            xpath = f'//*[@id="root"]/div[2]/div[2]/div/div/div[4]/div/div[3]/table/tbody/tr[{row_index}]/td[2]/a'
            report_click = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            human_like_move(driver, report_click)
            random_delay(0.2, 0.7)
            report_click.click()
            random_delay(0.2, 0.7)

            human_like_scroll(driver)
            random_delay(0.2, 0.7)
            
            download_bt = wait.until(EC.element_to_be_clickable((By.XPATH,
                '//*[@id="root"]/div[2]/div[2]/div/div/div[1]/div[2]/button')))
            human_like_move(driver, download_bt)
            random_delay(0.2, 0.7)
            download_bt.click()

            driver.back()
            random_delay(0.7, 1.5)

        except TimeoutException:
            if row_index == 1:
                print("[WARN] 1번 보고서 없음 → 다음 고객으로 이동")
                return False
            else:
                print(f"[WARN] row {row_index}에 보고서 없음 → 다음 row로 이동")
                continue
        except Exception as e:
            print(f"[ERROR] row {row_index} 처리 중 오류: {e}")
    return True


def change_customer_id(driver, wait, customer_id):
    """
    Customer ID 전환 시도 → 상단 표시된 ID가 바뀌었는지 비교 → 실패시 입력창 clear
    """
    previous_id = get_current_customer_display(driver)
    print(f"[INFO] 전환 전 표시 ID: {previous_id}")

    human_like_scroll(driver)
    random_delay(0.5, 1.0)
    re_cusid_input = wait.until(EC.element_to_be_clickable((
        By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[1]/div[2]/div[1]/button/div')))
    human_like_move(driver, re_cusid_input)
    random_delay(0.1, 0.3)
    re_cusid_input.click()
    cus_input = driver.find_element(By.CLASS_NAME, 'form-control')
    human_like_move(driver, cus_input)
    random_delay(0.5, 1.8)
    cus_input.click()
    cus_input.send_keys(Keys.TAB)
    random_delay(0.5, 1.8)

    human_like_scroll(driver)
    
    customer_input = driver.switch_to.active_element
    customer_input.clear()
    random_delay(0.5, 1.8)

    
    pyperclip.copy(customer_id)
    customer_input.send_keys(Keys.CONTROL, 'v')
    human_like_move(driver, customer_input)
    customer_input.send_keys(Keys.ENTER)
    random_delay(0.5, 1.8)

    new_id = get_current_customer_display(driver)
    print(f"[INFO] 전환 후 표시 ID: {new_id}")
    time.sleep(3)

    if new_id and (customer_id in new_id) and (new_id != previous_id):
        print(f"[SUCCESS] CustomerID {customer_id} 전환 성공!")
        return True
    else:
        print(f"[FAIL] CustomerID {customer_id} 전환 실패")

        try:
            print("[INFO] 잘못된 입력값 clear 시도")
            human_like_scroll(driver)
            random_delay(0.5, 1.0)
            re_cusid_input = wait.until(EC.element_to_be_clickable((
                By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[1]/div[1]/div[2]/div[1]/button/div')))
            human_like_move(driver, re_cusid_input)
            random_delay(0.1, 0.3)
            re_cusid_input.click()
            
            cus_input = driver.find_element(By.CLASS_NAME, 'form-control')
            cus_input.click()
            cus_input.send_keys(Keys.TAB)
            random_delay(0.5, 1.8)

            
            customer_input = driver.switch_to.active_element
            customer_input.clear()
            random_delay(1.3, 2.0)
            print("[INFO] 입력창 clear 완료")
        except Exception as e:
            print(f"[WARN] 입력창 clear 실패: {e}")

        return False

#  메인 실행
wait = WebDriverWait(driver, 15)

# 전환할 Customer ID 리스트
# customer_ids = ['2387950', '2910307', '825373']


for cid in customer_ids:
    print(f"\n=== [INFO] 전환 시도 → CustomerID: {cid} ===")
    
    # 1) Customer ID 전환
    if not change_customer_id(driver, wait, cid):
        print(f"[SKIP] {cid} 전환 실패 → 다운로드 건너뜀")
        continue
    
    # 2) 보고서 다운로드 시도
    got_reports = download_all_reports(driver, wait)
    if not got_reports:
        # 1번 보고서가 없어서 False가 돌아온 경우
        print(f"[SKIP] {cid} 1번 보고서 없음 → 다음 고객으로 이동")
        continue
    
    # 3) 다운로드 성공
    print(f"[DONE] {cid} 다운로드 완료")
    # → 여기서 추가 후속 처리 로직을 이어가시면 됩니다.


=== [INFO] 전환 시도 → CustomerID: 2387950 ===
[INFO] 전환 전 표시 ID: 2832953
[INFO] 전환 후 표시 ID: 2387950
[SUCCESS] CustomerID 2387950 전환 성공!
[INFO] 보고서 클릭 → row 1
[INFO] 보고서 클릭 → row 2
[INFO] 보고서 클릭 → row 3
[DONE] 2387950 다운로드 완료

=== [INFO] 전환 시도 → CustomerID: 2910307 ===
[INFO] 전환 전 표시 ID: 2387950
[INFO] 전환 후 표시 ID: 2910307
[SUCCESS] CustomerID 2910307 전환 성공!
[INFO] 보고서 클릭 → row 1
[WARN] 1번 보고서 없음 → 다음 고객으로 이동
[SKIP] 2910307 1번 보고서 없음 → 다음 고객으로 이동

=== [INFO] 전환 시도 → CustomerID: 825373 ===
[INFO] 전환 전 표시 ID: 2910307
[INFO] 전환 후 표시 ID: 825373
[SUCCESS] CustomerID 825373 전환 성공!
[INFO] 보고서 클릭 → row 1
[INFO] 보고서 클릭 → row 2
[INFO] 보고서 클릭 → row 3
[DONE] 825373 다운로드 완료
